In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import glob
import shutil

In [0]:
#shutil.copy("drive/My Drive/weather_turkey_processed.zip","./")
#!unzip weather_turkey_processed.zip

In [0]:
shutil.copy("drive/My Drive/RealTimeConsumption-01122014-09032020.csv","./")

'./RealTimeConsumption-01122014-09032020.csv'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

In [182]:
consumption_df=pd.read_csv("RealTimeConsumption-01122014-09032020.csv",encoding='latin-1')
consumption_df.columns=["Date","Hour","Consumption"]
consumption_df["Datetime"]=consumption_df["Date"]+" "+consumption_df["Hour"]
consumption_df["Consumption"]=consumption_df["Consumption"].apply(lambda x:x.replace(",","")).astype(float)
consumption_df["Datetime"]=pd.to_datetime(consumption_df["Datetime"])
consumption_df=consumption_df.loc[consumption_df["Datetime"]<pd.to_datetime("2020-03-04 00:00:00")]
consumption_df

,Date,Hour,Consumption,Datetime
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00
...,...,...,...,...
36571,03.03.2020,19:00,37936.79,2020-03-03 19:00:00
36572,03.03.2020,20:00,37553.37,2020-03-03 20:00:00
36573,03.03.2020,21:00,36581.73,2020-03-03 21:00:00
36574,03.03.2020,22:00,35434.36,2020-03-03 22:00:00


In [0]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
exp_eval_df=[]
def eval_function(name_,y_true, y_pred):
    print("RMSE: ",np.sqrt(mean_squared_error(y_true,y_pred)))
    #print("RMSLE: ",np.sqrt(mean_squared_log_error(y_true,y_pred)))
    print("MAE: ",mean_absolute_error(y_true,y_pred))
    print("MAPE: ",np.mean(np.abs((y_true - y_pred) / y_true)) * 100 )
    return name_, np.sqrt(mean_squared_error(y_true,y_pred)),mean_absolute_error(y_true,y_pred), np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#HW Exponential Smoothing without Special days and weekly Cycles

In [0]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [0]:
ExpSmt = ExponentialSmoothing(consumption_df["Consumption"], seasonal_periods=24, trend='add', seasonal='add').fit(use_boxcox=True)

In [136]:
exp_eval_df.append(eval_function("HW_ExpoSth",consumption_df["Consumption"],ExpSmt.fittedvalues))

RMSE:  646.3209311253985
MAE:  447.9737093609095
MAPE:  1.412794376192841


#HW Exponential Smoothing for Each Hour Including Weekly Cycles

In [0]:
ExpSmtArray=[]
MAPEs=[]
for hour in np.unique(consumption_df["Hour"]):
  Exp=ExponentialSmoothing(consumption_df.loc[consumption_df['Hour'] == hour]["Consumption"], seasonal_periods=7, trend='add', seasonal='add').fit(use_boxcox=True)
  pred_df=pd.DataFrame(list(zip(consumption_df.loc[consumption_df['Hour'] == hour]["Consumption"],Exp.fittedvalues)), columns =['real', 'pred']) 
  pred_df=pred_df.dropna()
  MAPEs.append(eval_function("HW_ExpoSth",pred_df["real"],pred_df["pred"])[3])
np.mean(MAPEs)


#HW Including Special Days by Regression Coefficients


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

In [0]:
special_days_path="drive/My Drive/ozel_gunler/*"
special_day_df=pd.DataFrame()
exception=[]
for i in tqdm(glob.glob(special_days_path)):  
    a=pd.read_csv(i)
    try:
        a["special_day"]=np.full(len(a),a.columns[0])
        a.columns=["date","special_day"]
        special_day_df=pd.concat([special_day_df, a])
    except ValueError:
        exception.append(i)
        pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [0]:
special_day_df["is"]=np.ones(len(special_day_df))
special_day_df=pd.pivot_table(special_day_df,index="date",columns="special_day").fillna(0).astype(int)
special_day_df.columns=['_'.join(i) for i in special_day_df.columns]
special_day_df=special_day_df.reset_index()

In [0]:
cons_lagged=consumption_df.copy()
cons_lagged["Consumption_2_lag"]=cons_lagged["Consumption"].shift(24*2)
cons_lagged["Consumption_7_lag"]=cons_lagged["Consumption"].shift(24*7)
cons_lagged["Hour"]=cons_lagged["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cons_lagged["Day"]=cons_lagged["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cons_lagged["Month"]=cons_lagged["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cons_lagged["weekday"]=pd.to_datetime(cons_lagged["Date"]).apply(lambda x: x.weekday()).astype(int)
special_day_df["date"]=pd.to_datetime(special_day_df["date"])
special_day_df["date"].dtype
cons_lagged=pd.merge(cons_lagged,special_day_df,left_on="Datetime",right_on="date",how="left").drop(columns="date").fillna(0)
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
cat_features=["Hour","Day","Month","weekday"]
linear_enc=OneHotEncoder()
one_hot_encoded=linear_enc.fit_transform(cons_lagged[cat_features]).toarray()
normal_feats=cons_lagged.columns.difference(cat_features).tolist()
cons_lagged=np.hstack([cons_lagged[normal_feats],one_hot_encoded])
cons_lagged=pd.DataFrame(cons_lagged,columns=normal_feats+np.concatenate([[cat_features[i]+"_"+str(j) for j in cats_]for i,cats_ in enumerate(linear_enc.categories_)]).tolist())
cons_lagged=cons_lagged.dropna()
cons_lagged

,Consumption,Consumption_2_lag,Consumption_7_lag,Date,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,...,Day_11,Day_12,Day_13,Day_14,Day_15,Day_16,Day_17,Day_18,Day_19,Day_20,Day_21,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,29590.9,0,0,31.12.2015,2015-12-31 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1,27785.4,0,0,31.12.2015,2015-12-31 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,26516.5,0,0,31.12.2015,2015-12-31 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,26091.6,0,0,31.12.2015,2015-12-31 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
4,25871.5,0,0,31.12.2015,2015-12-31 04:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36139,37936.8,35049.2,39283.7,03.03.2020,2020-03-03 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
36140,37553.4,34901.8,38791.4,03.03.2020,2020-03-03 20:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
36141,36581.7,34225.1,37588.9,03.03.2020,2020-03-03 21:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
36142,35434.4,33257.1,36290.6,03.03.2020,2020-03-03 22:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
train_df=cons_lagged.loc[cons_lagged["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cons_lagged.loc[cons_lagged["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]
from sklearn.linear_model import LinearRegression

cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values
model=LinearRegression()
model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
cons_lagged=pd.merge(consumption_df,special_day_df,left_on="Datetime",right_on="date",how="left").drop(columns="date").fillna(0)
sp_days_index=[i for i,col in enumerate(train_df[cols].columns) if col.startswith("is_")]

In [0]:
special_days_effect_dict=dict(zip(train_df[cols].columns[sp_days_index].tolist(),model.coef_[0][sp_days_index].tolist()))
for key_ in special_days_effect_dict.keys():
    cons_lagged[key_]=cons_lagged[key_]*special_days_effect_dict[key_]
#cons_lagged["Consumption"]=cons_lagged["Consumption"]-cons_lagged[list(special_days_effect_dict.keys())].sum(axis=1)
special_clmn=[i for i,col in enumerate(cons_lagged.columns) if col.startswith("is_")]
#cons_lagged.ix[:,107]
#sp_days_index
#cons_lagged.isnull().sum()

In [128]:
cons_lagged['HW_predictions']=ExponentialSmoothing(consumption_df["Consumption"], seasonal_periods=24, trend='add', seasonal='add').fit(use_boxcox=True).fittedvalues
cons_lagged_exp=cons_lagged.dropna()
#cons_lagged_exp['modified_pred_by_regression_coeff']=cons_lagged_exp['HW_predictions']+(cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index]).sum(axis=1)
cons_lagged_exp['modified_pred_by_regression_coeff']=cons_lagged_exp['HW_predictions']+(cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index]).max(axis=1)
cons_lagged_exp_test=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]
cons_lagged_exp_train=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
exp_eval_df.append(eval_function("HW_modified_by_regression",cons_lagged_exp_test["Consumption"],cons_lagged_exp_test['modified_pred_by_regression_coeff']))
#cons_lagged_exp[["Consumption","modified_pred_by_regression_coeff"]]
#cons_lagged['modified_pred_by_regression_coeff'].isnull().sum()#,cons_lagged['modified_pred_by_regression_coeff']

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:215: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


RMSE:  318133.58284070744
MAE:  23541.386728971513
MAPE:  77.06289887890478


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user

In [0]:
#np.count_nonzero((cons_lagged_exp.ix[:,special_clmn]*model.coef_[0][sp_days_index]).sum(axis=1))
cons_lagged_exp_test[["Consumption",'modified_pred_by_regression_coeff']]

#HW Including Special Days as Error Factor by the Previous Years 

In [0]:
cons_lagged_exp["HW_percent_errors"]=(cons_lagged_exp['HW_predictions']-cons_lagged_exp['Consumption'])/cons_lagged_exp['Consumption']
cons_lagged_exp_test=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]
cons_lagged_exp_train=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
#model=LinearRegression()
#model.fit(train_df[cols].values,train_df["Consumption"].values.reshape(-1, 1))
#cols=train_df.columns.difference(["Date","Consumption","Datetime"]).values
#cons_lagged_exp["HW_error"]
#Ex=ExponentialSmoothing(train_df["Consumption"], seasonal_periods=24, trend='add', seasonal='add').fit(use_boxcox=True)
#scaled_holiday_errors=(Ex.fittedvalues-train_df.loc[consumption_df["Datetime"]<["Consumption"])/train_df["Consumption"]
model_by_error=LinearRegression(fit_intercept=False)
model_by_error.fit(cons_lagged_exp_train.ix[:,special_clmn].values,cons_lagged_exp_train["HW_percent_errors"].values.reshape(-1, 1))
cons_lagged_exp_test['modified_by_prev_error']=cons_lagged_exp_test['HW_predictions']*(1-(cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0]).max(axis=1))
exp_eval_df.append(eval_function("HW_modified_by_error",cons_lagged_exp_test["Consumption"],cons_lagged_exp_test['modified_by_prev_error']))
#(cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0]).max(axis=1)
#cons_lagged_exp_test=cons_lagged_exp.loc[cons_lagged_exp["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]model_by_error.coef_
#cons_lagged_exp.ix[:,special_clmn]

In [219]:
np.nonzero((cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0])['is_arife'])[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


array([ 480, 5328])

In [229]:
#cons_lagged_exp_test['HW_predictions']*
#cons_lagged_exp_test[np.nonzero((cons_lagged_exp_test.ix[:,special_clmn]*model_by_error.coef_[0])['is_arife'])]

cons_lagged_exp_test.iloc[480]

Date                                          06.03.2019
Hour                                               00:00
Consumption                                      31516.8
Datetime                             2019-06-03 00:00:00
is_annelergunu                                         0
is_arife                                         3108.39
is_babalargunu                                         0
is_cocukbayrami                                        0
is_cumhuriyet                                         -0
is_elections                                           0
is_examination                                         0
is_genclikbayrami                                     -0
is_holiday                                      -2153.05
is_iscibayrami                                         0
is_kadinlargunu                                        0
is_kandil                                             -0
is_kurbanbayrami                                      -0
is_kurbanbayramilkgun          

In [197]:
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged_exp_test["Date"]=="11.08.2019"]["Consumption"],cons_lagged_exp_test.loc[cons_lagged_exp_test['Date']=="11.08.2019"]['modified_by_prev_error'])
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged_exp_test["Date"]=="29.11.2019"]["Consumption"],cons_lagged_exp_test.loc[cons_lagged_exp_test['Date']=="29.11.2019"]['modified_by_prev_error'])

RMSE:  1635.3531257507652
MAE:  1199.3832340265842
MAPE:  4.686318505811885
RMSE:  539.4578995073159
MAE:  423.815928634306
MAPE:  1.1585038298968189


('HW_modified_by_error',
 539.4578995073159,
 423.815928634306,
 1.1585038298968189)

In [198]:
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged["Date"]=="11.08.2019"]["Consumption"],cons_lagged.loc[cons_lagged['Date']=="11.08.2019"]['HW_predictions'])
eval_function("HW_modified_by_error",cons_lagged_exp_test.loc[cons_lagged["Date"]=="29.11.2019"]["Consumption"],cons_lagged.loc[cons_lagged['Date']=="29.11.2019"]['HW_predictions'])

RMSE:  1636.2998414239462
MAE:  1205.3475423066493
MAPE:  4.706172366712435
RMSE:  539.4318065258644
MAE:  423.36845960858153
MAPE:  1.1570989172735997


('HW_modified_by_error',
 539.4318065258644,
 423.36845960858153,
 1.1570989172735997)

#HW by Interpolated Data and Naive Approach for Special Days(mantıklı olmayabilir)